In [1]:
import pandas as pd
import time
import random
import math
import mmh3

In [2]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB):  # DBLP
        if index2 > len(df2) - 1:
            return True
        
        rr = df2.iloc[index2]
        ncid = rr["TID2"]
        title = rr["title"]
        length = rr["length"]
        artist = rr["artist"]
        album = rr["album"]
        srec = title + " " + length + " " + str(artist) + " " + str(album)
    
        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
                    
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == ncid:
                        tp += 1
                        break
            else:
                fp += 1

    return False

# Descarte Simples

In [9]:
df1 = pd.read_csv("musicbrainz-200-A01.csv.dapo", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("music_brainz-simple-mutated.csv", sep=",", encoding="utf-8", keep_default_na=False)

truth = pd.read_csv("ground_truth_music_brainz.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    novoId = r["novoidmusic1"]
    antigoId = r["antigoidmusic2"]
    if antigoId in truthD:
        ids = truthD[antigoId]
        ids.append(novoId)
    else:
        truthD[antigoId] = [novoId]

t = 0.5
p1 = (t) ** 8
L = math.ceil(math.log(0.1) / math.log(1 - p1))
# print("Minhash LSH=", L)
TP = 193471
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
indices = [random.randint(0, L) for i in range(L1)]
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0
total_tempermanencia = 0
total_entidades_descartadas = 0

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):  # Scholar
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1]
        ncid = rr["TID"]
        title = rr["title"]
        length = rr["length"]
        artist = rr["artist"]
        album = rr["album"]
        srec = title + " " + length + " " + str(artist) + " " + str(album)
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                if len(ids) < w:
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    entidade_removida = ids.pop(0)
                    tempo_insercao = ids_igual.pop(0)
                    
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura) 

                    # Calcula tempo de permanência corretamente
                    tempo_permanencia = tempoQueFoiInseridoNaEstrutura - tempo_insercao
                    total_tempermanencia += tempo_permanencia
                    total_entidades_descartadas += 1
            else:
                d[key] = [ncid]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]


        tempoQueFoiInseridoNaEstrutura += 1


    end = time.time()
    blockingTime += (end - st)
    st = time.time()
    #You can use either method matching() or topK()  
    
    termination = matching()  
    #termination = topK()
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

if total_entidades_descartadas > 0:
    tempo_medio_permanencia = total_tempermanencia / total_entidades_descartadas
    print(f"Tempo médio de permanência das entidades: {tempo_medio_permanencia:.2f}")
else:
    print("Nenhuma entidade foi descartada. Tempo médio = 0")

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)


L= 116 L1= 50
Tempo médio de permanência das entidades: 20858.32
blocking time (in mins) 6.269537695248922
matching time (in mins) 21.914513127009073
TP= 20578 Recall= 0.10636219381716124 Precision= 0.6592343424635592 pairsNo= 41629


# Descarte de Entidades

In [10]:
def elimina_elementos_dentro_dictB(array_ids, array_tempos, limite,
                                   tempo_atual,
                                   stats):
    """
    array_ids: lista de ids
    array_tempos: tempos de inserção
    limite: descartar tudo com tempo < limite
    tempo_atual: tempo global
    stats: dicionário com acumuladores
    """

    indice = 0
    for tempo_inserido in array_tempos:
        if tempo_inserido < limite:
            # calcular tempo de permanência
            permanencia = tempo_atual - tempo_inserido
            stats["soma_tempos"] += permanencia
            stats["total_descartados"] += 1
            indice += 1
        else:
            break

    # descartar prefixo
    del array_ids[:indice]
    del array_tempos[:indice]

In [11]:
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
indices = [random.randint(0, L) for i in range(L1)]
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0
total_tempermanencia = 0
total_entidades_descartadas = 0
acompanhamentoIndicePorBloco = {}
elementos_para_descarte = 50  # Número de elementos a serem descartados quando o limite for atingido

stats = {
    "soma_tempos": 0,
    "total_descartados": 0
}

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):  # Scholar
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1]
        ncid = rr["TID"]
        title = rr["title"]
        length = rr["length"]
        artist = rr["artist"]
        album = rr["album"]
        srec = title + " " + length + " " + str(artist) + " " + str(album)
        key = ""
    
        for l in range(L):
                key = str(str_to_MinHash(srec.lower(), 2, l))
                d = dictB[l]
                d_igual = dictB_igual[l]
                if key in d:
                    ids = d[key]
                    ids_igual = d_igual[key]
                    if len(ids) < w:
                        ids.append(ncid)
                        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                    else:
                        elementoAtualParaDescarte = acompanhamentoIndicePorBloco[(key, l)]
                        elimina_elementos_dentro_dictB(ids,ids_igual,elementoAtualParaDescarte,tempoQueFoiInseridoNaEstrutura,stats)
                        acompanhamentoIndicePorBloco[(key, l)] = elementoAtualParaDescarte + elementos_para_descarte
                        
                        ids.append(ncid)
                        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    d[key] = [ncid]
                    d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                    acompanhamentoIndicePorBloco[(key, l)] = elementos_para_descarte

        tempoQueFoiInseridoNaEstrutura += 1


    end = time.time()
    blockingTime += (end - st)
    st = time.time()
    #You can use either method matching() or topK()  
    
    termination = matching()  
    #termination = topK()
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA
    
if stats["total_descartados"] > 0:
    tempo_medio = stats["soma_tempos"] / stats["total_descartados"]
else:
    tempo_medio = 0

print("Tempo médio de permanência:", tempo_medio)
print("Entidades descartadas:", stats["total_descartados"])

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)


Tempo médio de permanência: 19253.744288339305
Entidades descartadas: 16276352
blocking time (in mins) 5.724148992697398
matching time (in mins) 20.484394856293996
TP= 20578 Recall= 0.10636219381716124 Precision= 0.6560607026716827 pairsNo= 41753


# Descarte de Blocos

In [12]:
def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual, stats, tempo_atual):
    """
    array_para_descarte           = lista de IDs do bloco
    array_para_descarte_igual     = lista de timestamps de inserção
    stats["soma_tempos"]          = soma dos tempos de permanência
    stats["total_descartados"]    = contagem total de descartados
    tempo_atual                   = tempo global no momento do descarte
    """

    # Para cada elemento descartado, soma tempo de permanência
    for tempo_inserido in array_para_descarte_igual:
        permanencia = tempo_atual - tempo_inserido
        stats["soma_tempos"] += permanencia
        stats["total_descartados"] += 1

    # Limpa o bloco inteiro
    array_para_descarte.clear()
    array_para_descarte_igual.clear()



dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]

tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0

# novo dicionário de estatísticas
stats = {
    "soma_tempos": 0,
    "total_descartados": 0
}

tamanhoDosBlocos = {}

while True:
    st = time.time()

    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break

        rr = df1.iloc[index1]
        ncid = rr["TID"]
        title = rr["title"]
        length = rr["length"]
        artist = rr["artist"]
        album = rr["album"]

        srec = title + " " + length + " " + str(artist) + " " + str(album)

        for l in range(L):

            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]

            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]

                if len(ids) < tamanho_atual:
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    # descarta bloco inteiro
                    elimina_elementos_dentro_dictB(
                        ids, ids_igual, stats, tempoQueFoiInseridoNaEstrutura
                    )

                    # reinsere o elemento atual após descarte
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)

            else:
                d[key] = [ncid]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                tamanhoDosBlocos[(key, l)] = w

        tempoQueFoiInseridoNaEstrutura += 1

    end = time.time()
    blockingTime += (end - st)

    st = time.time()
    termination = matching()
    end = time.time()
    matchingTime += (end - st)

    if termination:
        break

    nbS += offsetB
    naS += offsetA


# ================================
#       RESULTADOS FINAIS
# ================================

if stats["total_descartados"] > 0:
    tempo_medio = stats["soma_tempos"] / stats["total_descartados"]
else:
    tempo_medio = 0

print("Tempo médio de permanência:", tempo_medio)
print("Entidades descartadas:", stats["total_descartados"])

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)

if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)


Tempo médio de permanência: 13260.215179659182
Entidades descartadas: 18485000
blocking time (in mins) 5.5956775943438215
matching time (in mins) 11.648817042509714
TP= 20578 Recall= 0.10636219381716124 Precision= 0.8301932464598378 pairsNo= 28766
